In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Installing libraries

In [2]:
!pip install textattack
!pip install -U gensim==4.0.0
!pip install python-Levenshtein

     |████████████████████████████████| 358kB 21.0MB/s 
     |████████████████████████████████| 20.1MB 1.3MB/s 
     |████████████████████████████████| 61kB 9.9MB/s 
     |████████████████████████████████| 2.1MB 51.9MB/s 
     |████████████████████████████████| 102kB 12.7MB/s 
     |████████████████████████████████| 778kB 50.6MB/s 
     |████████████████████████████████| 337kB 56.2MB/s 
     |████████████████████████████████| 225kB 28.3MB/s 
     |████████████████████████████████| 901kB 50.0MB/s 
     |████████████████████████████████| 3.3MB 50.0MB/s 
     |████████████████████████████████| 19.7MB 171kB/s 
     |████████████████████████████████| 983kB 49.2MB/s 
     |████████████████████████████████| 71kB 11.3MB/s 
     |████████████████████████████████| 1.2MB 46.3MB/s 
     |████████████████████████████████| 798kB 47.1MB/s 
     |████████████████████████████████| 112kB 54.1MB/s 
     |████████████████████████████████| 245kB 59.2MB/s 
  Created wheel for word2number: filename=word2numb

     |████████████████████████████████| 23.9MB 121kB/s 
  Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0
     |████████████████████████████████| 51kB 7.2MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149809 sha256=9ead9338add20707b28279302e78b92174f57238c11bba993524d62f99781bd0
  Stored in directory: /root/.cache/pip/wheels/b3/26/73/4b48503bac73f01cf18e52cd250947049a7f339e940c5df8fc
Successfully built python-Levenshtein


In [3]:
import gensim.downloader as api
from gensim.models.word2vec import Word2Vec
from gensim.models.fasttext import FastText
from gensim.models import Phrases
from gensim.test.utils import get_tmpfile
import transformers
from textattack.models.tokenizers import AutoTokenizer
from textattack.models.wrappers import HuggingFaceModelWrapper
from textattack.transformations import WordSwap
from textattack.search_methods import GreedySearch, GreedyWordSwapWIR
from textattack.constraints.overlap import LevenshteinEditDistance
from textattack.constraints.pre_transformation import RepeatModification, StopwordModification
from textattack.search_methods import ParticleSwarmOptimization
from textattack.constraints.grammaticality import PartOfSpeech
from textattack.shared import Attack
from textattack.goal_functions import UntargetedClassification
from textattack.datasets import HuggingFaceDataset
from collections import deque

import textattack
import tqdm
import time
from IPython.display import display, HTML

textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


textattack: Downloading https://textattack.s3.amazonaws.com/word_embeddings/paragramcf.
100%|██████████| 481M/481M [00:08<00:00, 57.4MB/s]
textattack: Unzipping file /root/.cache/textattack/tmp0pq7h9k1.zip to /root/.cache/textattack/word_embeddings/paragramcf.
textattack: Successfully saved word_embeddings/paragramcf to cache.


In [4]:
# import gensim.downloader as api
# corpus = api.load('text8')
# from gensim.models.fasttext import FastText
# model1 = FastText(corpus, sample = 0, sg = 1)

In [5]:
# import gensim.downloader as api
# corpus = api.load('text8')
# from gensim.models.word2vec import Word2Vec
# model2 = Word2Vec(corpus, sample = 0, sg = 1)

# Loading FastText, Word2Vec and Glove

In [6]:
from gensim.test.utils import get_tmpfile
from gensim.models.fasttext import FastText
#fname = get_tmpfile("/models/fasttext.model") 
model1 = FastText.load("/content/drive/MyDrive/models/fasttext.model") # Change this location

In [7]:
from gensim.test.utils import get_tmpfile
from gensim.models.word2vec import Word2Vec
#fname1 = get_tmpfile("/models/word2vec.model") 
model2 = Word2Vec.load("/content/drive/MyDrive/models/word2vec.model") # Change this location

In [8]:
from gensim.models import KeyedVectors
model3=KeyedVectors.load('/content/drive/MyDrive/models/gloveKeyVectors.kv')

Example demonstration

In [9]:
print(model1.wv.most_similar('cat'))
print(model2.wv.most_similar('cat'))
print(model3.most_similar('cat'))

[('bobcat', 0.759185791015625), ('dog', 0.7208123803138733), ('foxglove', 0.7110776901245117), ('pig', 0.7088143229484558), ('goose', 0.7069101333618164), ('cats', 0.702387273311615), ('beak', 0.7008379697799683), ('catfish', 0.6987316608428955), ('foxhound', 0.696578860282898), ('catnip', 0.6960986852645874)]
[('dog', 0.7193913459777832), ('squirrel', 0.696162760257721), ('leopardus', 0.6945451498031616), ('prionailurus', 0.688106894493103), ('albino', 0.6880608201026917), ('bird', 0.6836060285568237), ('eared', 0.6803243160247803), ('rhinoceros', 0.6769237518310547), ('hyena', 0.6756084561347961), ('badger', 0.6752869486808777)]
[('cats', 0.7006820440292358), ('dog', 0.698471188545227), ('dogs', 0.6442825198173523), ('tervueren', 0.619421660900116), ('laekenois', 0.5972026586532593), ('lica', 0.5956287980079651), ('feral', 0.5899650454521179), ('goat', 0.5861616134643555), ('fish', 0.5746133327484131), ('malinois', 0.5737614631652832)]


In [10]:
if 'cat' in model3.key_to_index:
  print('present')

present


# Distilbert Base Cased-QQP

In [11]:
# Import the model
import transformers
from textattack.models.tokenizers import AutoTokenizer
from textattack.models.wrappers import HuggingFaceModelWrapper

model = transformers.AutoModelForSequenceClassification.from_pretrained("textattack/albert-base-v2-rotten-tomatoes")
tokenizer = AutoTokenizer("textattack/albert-base-v2-rotten-tomatoes")
# Change these

model_wrapper = HuggingFaceModelWrapper(model, tokenizer)

# Framing fasttext attack

In [12]:
from textattack.transformations import WordSwap

class Swapper(WordSwap): # For fasttext
    """ Transforms an input by replacing any word with its most similar counterpart
    """

    # We don't need a constructor, since our class doesn't require any parameters.

    def _get_replacement_words(self, word):
        for i in range(10):
            if word.lower() in model1.wv.most_similar(word)[i][0].lower():
                continue # Don't return the exact word
            elif word.isupper():
                return [model1.wv.most_similar(word)[i][0].upper()] # Preserving case
            elif word[0].isupper():
                return [model1.wv.most_similar(word)[i][0].capitalize()] # Preserving Capitalization in words
            else:
                return [model1.wv.most_similar(word)[i][0]]
            
        return [model1.wv.most_similar(word)[0][0]]

In [13]:
def attack_stats(attack, dataset, num_of_examples=25):
  print(attack)
  num_examples = num_of_examples
  num_remaining_attacks = num_examples
  pbar = tqdm.tqdm(total=num_remaining_attacks, smoothing=0)

  worklist = deque(range(0, num_examples))
  worklist_tail = worklist[-1]

  attack_log_manager = textattack.loggers.AttackLogManager()

  load_time = time.time()
        
  num_results = 0
  num_failures = 0
  num_successes = 0
  for result in attack.attack_dataset(dataset, indices=worklist):
      result_html_str = result.__str__(color_method="html").replace("\n\n", "<br>")
      display(HTML(result_html_str))
      attack_log_manager.log_result(result)
            
      if not isinstance(result, textattack.attack_results.SkippedAttackResult):
          pbar.update(1)
      else:
          worklist_tail += 1
          pbar.update(1)
          worklist.append(worklist_tail)

      num_results += 1

      if (type(result) == textattack.attack_results.SuccessfulAttackResult
        or type(result) == textattack.attack_results.MaximizedAttackResult):
          num_successes += 1
                
      if type(result) == textattack.attack_results.FailedAttackResult:
          num_failures += 1
      pbar.set_description(
          "[Succeeded / Failed / Total] {} / {} / {}".format(
            num_successes, num_failures, num_results
        )
      )

  pbar.close()

  attack_log_manager.enable_stdout()
  attack_log_manager.log_summary()
  attack_log_manager.flush()
        
  textattack.shared.logger.info(f"Attack time: {time.time() - load_time}s")

### FastText, GreedySearch, RepeatModification, StopwordModification

In [14]:

# We're going to use our word swap class as the attack transformation.
transformation = Swapper()

constraints = [RepeatModification(),
               StopwordModification()]

search_method = GreedySearch()

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

dataset = HuggingFaceDataset("rotten_tomatoes", None, "test") # Change the dataset

textattack: Unknown if model of class <class 'transformers.models.albert.modeling_albert.AlbertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


Using custom data configuration default


textattack: Loading datasets dataset rotten_tomatoes, split test.


Dataset rotten_tomatoes_movie_review downloaded and prepared to /root/.cache/huggingface/datasets/rotten_tomatoes_movie_review/default/1.0.0/9c411f7ecd9f3045389de0d9ce984061a1056507703d2e3183b1ac1a90816e4d. Subsequent calls will reuse this data.


In [15]:
attack_stats(attack, dataset, 25)

  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): GreedySearch
  (goal_function):  UntargetedClassification
  (transformation):  Swapper
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


[Succeeded / Failed / Total] 1 / 0 / 1:   4%|▍         | 1/25 [00:01<00:25,  1.07s/it]

[Succeeded / Failed / Total] 1 / 1 / 2:   8%|▊         | 2/25 [00:01<00:14,  1.61it/s]

[Succeeded / Failed / Total] 1 / 1 / 3:  12%|█▏        | 3/25 [00:01<00:09,  2.36it/s]

...

[Succeeded / Failed / Total] 19 / 4 / 29: : 29it [00:21,  1.36it/s]

[Succeeded / Failed / Total] 20 / 4 / 30: : 30it [00:22,  1.33it/s]

[Succeeded / Failed / Total] 21 / 4 / 31: : 31it [00:23,  1.33it/s]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 21     |
| Number of failed attacks:     | 4      |
| Number of skipped attacks:    | 6      |
| Original accuracy:            | 80.65% |
| Accuracy under attack:        | 12.9%  |
| Attack success rate:          | 84.0%  |
| Average perturbed word %:     | 12.4%  |
| Average num. words per input: | 18.74  |
| Avg num queries:              | 36.56  |
+-------------------------------+--------+


textattack: Attack time: 23.370148420333862s


# Framing word2vec attack

In [16]:
class Swapper_(WordSwap): # For word2vec
    """ Transforms an input by replacing any word with its most similar counterpart
    """

    # We don't need a constructor, since our class doesn't require any parameters.

    def _get_replacement_words(self, word):
        """ Returns 'banana', no matter what 'word' was originally.

            Returns a list with one item, since `_get_replacement_words` is intended to
                return a list of candidate replacement words.
        """
        if word in model2.wv.key_to_index:
            for i in range(10):
                if word.lower() in model2.wv.most_similar(word)[i][0].lower():
                    continue
                elif word.isupper():
                    return [model2.wv.most_similar(word)[i][0].upper()]
                elif word[0].isupper():
                    return [model2.wv.most_similar(word)[i][0].capitalize()]
                else:
                    return [model2.wv.most_similar(word)[i][0]]
        else:
            return [word]

### Word2vec, GreedySearch, RepeatModification, StopwordModification

In [17]:
# We're going to use our word swap class as the attack transformation.
transformation = Swapper_()

constraints = [RepeatModification(),
               StopwordModification()]

search_method = GreedySearch()

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)


textattack: Unknown if model of class <class 'transformers.models.albert.modeling_albert.AlbertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [18]:
print(attack)

Attack(
  (search_method): GreedySearch
  (goal_function):  UntargetedClassification
  (transformation):  Swapper_
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


In [19]:
attack_stats(attack, dataset, 25)

  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): GreedySearch
  (goal_function):  UntargetedClassification
  (transformation):  Swapper_
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


[Succeeded / Failed / Total] 1 / 0 / 1:   4%|▍         | 1/25 [00:00<00:07,  3.11it/s]

[Succeeded / Failed / Total] 1 / 1 / 2:   8%|▊         | 2/25 [00:00<00:05,  4.05it/s]

[Succeeded / Failed / Total] 1 / 1 / 3:  12%|█▏        | 3/25 [00:00<00:03,  5.76it/s]

...

[Succeeded / Failed / Total] 17 / 6 / 29: : 29it [00:15,  1.83it/s]

[Succeeded / Failed / Total] 17 / 7 / 30: : 30it [00:17,  1.75it/s]

[Succeeded / Failed / Total] 18 / 7 / 31: : 31it [00:17,  1.78it/s]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 18     |
| Number of failed attacks:     | 7      |
| Number of skipped attacks:    | 6      |
| Original accuracy:            | 80.65% |
| Accuracy under attack:        | 22.58% |
| Attack success rate:          | 72.0%  |
| Average perturbed word %:     | 12.78% |
| Average num. words per input: | 18.74  |
| Avg num queries:              | 29.48  |
+-------------------------------+--------+


textattack: Attack time: 17.415359258651733s


### Framing Glove attack

In [20]:
from textattack.transformations import WordSwap

class Swapper__(WordSwap): # For glove
    """ Transforms an input by replacing any word with its most similar counterpart
    """

    # We don't need a constructor, since our class doesn't require any parameters.

    def _get_replacement_words(self, word):
        """ Returns 'banana', no matter what 'word' was originally.

            Returns a list with one item, since `_get_replacement_words` is intended to
                return a list of candidate replacement words.
        """
        if word in model3.key_to_index:
            for i in range(10):
                if word.lower() in model3.most_similar(word)[i][0].lower():
                    continue
                elif word.isupper():
                    return [model3.most_similar(word)[i][0].upper()]
                elif word[0].isupper():
                    return [model3.most_similar(word)[i][0].capitalize()]
                else:
                    return [model3.most_similar(word)[i][0]]
        else:
            return [word]

### Glove, GreedySearch, RepeatModification, StopwordModification

In [21]:
# We're going to use our word swap class as the attack transformation.
transformation = Swapper__()

constraints = [RepeatModification(),
               StopwordModification()]

search_method = GreedySearch()

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

textattack: Unknown if model of class <class 'transformers.models.albert.modeling_albert.AlbertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [22]:
print(attack)

Attack(
  (search_method): GreedySearch
  (goal_function):  UntargetedClassification
  (transformation):  Swapper__
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


In [23]:
attack_stats(attack, dataset, 25)

  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): GreedySearch
  (goal_function):  UntargetedClassification
  (transformation):  Swapper__
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


[Succeeded / Failed / Total] 1 / 0 / 1:   4%|▍         | 1/25 [00:00<00:14,  1.71it/s]

[Succeeded / Failed / Total] 2 / 0 / 2:   8%|▊         | 2/25 [00:00<00:09,  2.47it/s]

[Succeeded / Failed / Total] 2 / 0 / 3:  12%|█▏        | 3/25 [00:00<00:06,  3.59it/s]

...

[Succeeded / Failed / Total] 20 / 3 / 29: : 29it [00:25,  1.16it/s]

[Succeeded / Failed / Total] 20 / 4 / 30: : 30it [00:27,  1.10it/s]

[Succeeded / Failed / Total] 21 / 4 / 31: : 31it [00:27,  1.12it/s]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 21     |
| Number of failed attacks:     | 4      |
| Number of skipped attacks:    | 6      |
| Original accuracy:            | 80.65% |
| Accuracy under attack:        | 12.9%  |
| Attack success rate:          | 84.0%  |
| Average perturbed word %:     | 14.99% |
| Average num. words per input: | 18.74  |
| Avg num queries:              | 27.84  |
+-------------------------------+--------+


textattack: Attack time: 27.803630828857422s


### FastText, GreedyWordSwapWIR, RepeatModification, StopwordModification

In [24]:
# We're going to use our word swap class as the attack transformation.
transformation = Swapper()

constraints = [RepeatModification(),
               StopwordModification()]

search_method = GreedyWordSwapWIR()

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

textattack: Unknown if model of class <class 'transformers.models.albert.modeling_albert.AlbertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [25]:
print(attack)

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


In [26]:
attack_stats(attack, dataset, 25)

  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


[Succeeded / Failed / Total] 1 / 0 / 1:   4%|▍         | 1/25 [00:00<00:09,  2.54it/s]

[Succeeded / Failed / Total] 1 / 1 / 2:   8%|▊         | 2/25 [00:00<00:06,  3.41it/s]

[Succeeded / Failed / Total] 1 / 1 / 3:  12%|█▏        | 3/25 [00:00<00:04,  4.92it/s]

...

[Succeeded / Failed / Total] 19 / 4 / 29: : 29it [00:10,  2.70it/s]

[Succeeded / Failed / Total] 20 / 4 / 30: : 30it [00:11,  2.62it/s]

[Succeeded / Failed / Total] 21 / 4 / 31: : 31it [00:11,  2.59it/s]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 21     |
| Number of failed attacks:     | 4      |
| Number of skipped attacks:    | 6      |
| Original accuracy:            | 80.65% |
| Accuracy under attack:        | 12.9%  |
| Attack success rate:          | 84.0%  |
| Average perturbed word %:     | 17.12% |
| Average num. words per input: | 18.74  |
| Avg num queries:              | 24.0   |
+-------------------------------+--------+


textattack: Attack time: 11.995541095733643s


### Word2Vec, GreedyWordSwapWIR, RepeatModification, StopwordModification

In [27]:
# We're going to use our word swap class as the attack transformation.
transformation = Swapper_()

constraints = [RepeatModification(),
               StopwordModification()]

search_method = GreedyWordSwapWIR()

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)


textattack: Unknown if model of class <class 'transformers.models.albert.modeling_albert.AlbertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [28]:
print(attack)

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper_
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


In [29]:
attack_stats(attack, dataset, 25)

  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper_
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


[Succeeded / Failed / Total] 1 / 0 / 1:   4%|▍         | 1/25 [00:00<00:08,  2.71it/s]

[Succeeded / Failed / Total] 1 / 1 / 2:   8%|▊         | 2/25 [00:00<00:06,  3.67it/s]

[Succeeded / Failed / Total] 1 / 1 / 3:  12%|█▏        | 3/25 [00:00<00:04,  5.25it/s]

...

[Succeeded / Failed / Total] 15 / 8 / 29: : 29it [00:10,  2.89it/s]

[Succeeded / Failed / Total] 15 / 9 / 30: : 30it [00:10,  2.79it/s]

[Succeeded / Failed / Total] 16 / 9 / 31: : 31it [00:11,  2.77it/s]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 16     |
| Number of failed attacks:     | 9      |
| Number of skipped attacks:    | 6      |
| Original accuracy:            | 80.65% |
| Accuracy under attack:        | 29.03% |
| Attack success rate:          | 64.0%  |
| Average perturbed word %:     | 11.84% |
| Average num. words per input: | 18.74  |
| Avg num queries:              | 23.12  |
+-------------------------------+--------+


textattack: Attack time: 11.202688932418823s


### Glove, GreedyWordSwapWIR RepeatModification, StopwordModification

In [30]:
# We're going to use our word swap class as the attack transformation.
transformation = Swapper__()

constraints = [RepeatModification(),
               StopwordModification()]

search_method = GreedyWordSwapWIR()

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

textattack: Unknown if model of class <class 'transformers.models.albert.modeling_albert.AlbertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [31]:
print(attack)

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper__
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


In [32]:
attack_stats(attack, dataset, 25)

  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper__
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


[Succeeded / Failed / Total] 1 / 0 / 1:   4%|▍         | 1/25 [00:00<00:09,  2.53it/s]

[Succeeded / Failed / Total] 2 / 0 / 2:   8%|▊         | 2/25 [00:00<00:06,  3.43it/s]

[Succeeded / Failed / Total] 2 / 0 / 3:  12%|█▏        | 3/25 [00:00<00:04,  4.95it/s]

...

[Succeeded / Failed / Total] 20 / 3 / 29: : 29it [00:11,  2.63it/s]

[Succeeded / Failed / Total] 20 / 4 / 30: : 30it [00:11,  2.52it/s]

[Succeeded / Failed / Total] 21 / 4 / 31: : 31it [00:12,  2.50it/s]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 21     |
| Number of failed attacks:     | 4      |
| Number of skipped attacks:    | 6      |
| Original accuracy:            | 80.65% |
| Accuracy under attack:        | 12.9%  |
| Attack success rate:          | 84.0%  |
| Average perturbed word %:     | 16.15% |
| Average num. words per input: | 18.74  |
| Avg num queries:              | 22.56  |
+-------------------------------+--------+


textattack: Attack time: 12.390251159667969s


### FastText, ParticleSwarmOptimization, RepeatModification, StopwordModification

In [33]:
from textattack.search_methods import ParticleSwarmOptimization


# We're going to use our word swap class as the attack transformation.
transformation = Swapper()

constraints = [RepeatModification(),
               StopwordModification()]

search_method = ParticleSwarmOptimization(pop_size=10,max_iters=5,post_turn_check=True)

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

textattack: Unknown if model of class <class 'transformers.models.albert.modeling_albert.AlbertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [34]:
print(attack)

Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  10
    (max_iters):  5
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


In [35]:
attack_stats(attack, dataset, 25)

  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  10
    (max_iters):  5
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


[Succeeded / Failed / Total] 1 / 0 / 1:   4%|▍         | 1/25 [00:02<01:06,  2.76s/it]

[Succeeded / Failed / Total] 1 / 1 / 2:   8%|▊         | 2/25 [00:03<00:35,  1.54s/it]

[Succeeded / Failed / Total] 1 / 1 / 3:  12%|█▏        | 3/25 [00:03<00:22,  1.03s/it]

...

[Succeeded / Failed / Total] 19 / 4 / 29: : 29it [01:22,  2.85s/it]

[Succeeded / Failed / Total] 20 / 4 / 30: : 30it [01:27,  2.90s/it]

[Succeeded / Failed / Total] 21 / 4 / 31: : 31it [01:30,  2.92s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 21     |
| Number of failed attacks:     | 4      |
| Number of skipped attacks:    | 6      |
| Original accuracy:            | 80.65% |
| Accuracy under attack:        | 12.9%  |
| Attack success rate:          | 84.0%  |
| Average perturbed word %:     | 16.1%  |
| Average num. words per input: | 18.74  |
| Avg num queries:              | 174.08 |
+-------------------------------+--------+


textattack: Attack time: 90.66420149803162s


### Word2Vec, ParticleSwarmOptimization, RepeatModification, StopwordModification

In [36]:
# We're going to use our word swap class as the attack transformation.
transformation = Swapper_()

constraints = [RepeatModification(),
               StopwordModification()]

search_method = ParticleSwarmOptimization(pop_size=10,max_iters=5,post_turn_check=True)

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

textattack: Unknown if model of class <class 'transformers.models.albert.modeling_albert.AlbertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [37]:
print(attack)

Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  10
    (max_iters):  5
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper_
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


In [38]:
attack_stats(attack, dataset, 25)

  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  10
    (max_iters):  5
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper_
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


[Succeeded / Failed / Total] 1 / 0 / 1:   4%|▍         | 1/25 [00:00<00:09,  2.45it/s]

[Succeeded / Failed / Total] 1 / 1 / 2:   8%|▊         | 2/25 [00:00<00:08,  2.71it/s]

[Succeeded / Failed / Total] 1 / 1 / 3:  12%|█▏        | 3/25 [00:00<00:05,  3.95it/s]

...

[Succeeded / Failed / Total] 17 / 6 / 29: : 29it [00:45,  1.58s/it]

[Succeeded / Failed / Total] 17 / 7 / 30: : 30it [00:51,  1.72s/it]

[Succeeded / Failed / Total] 18 / 7 / 31: : 31it [00:51,  1.67s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 18     |
| Number of failed attacks:     | 7      |
| Number of skipped attacks:    | 6      |
| Original accuracy:            | 80.65% |
| Accuracy under attack:        | 22.58% |
| Attack success rate:          | 72.0%  |
| Average perturbed word %:     | 13.53% |
| Average num. words per input: | 18.74  |
| Avg num queries:              | 119.8  |
+-------------------------------+--------+


textattack: Attack time: 51.76456570625305s


### Glove, ParticleSwarmOptimization, RepeatModification, StopwordModification

In [39]:
# We're going to use our word swap class as the attack transformation.
transformation = Swapper__()

constraints = [RepeatModification(),
               StopwordModification()]

search_method = ParticleSwarmOptimization(pop_size=10,max_iters=5,post_turn_check=True)

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

textattack: Unknown if model of class <class 'transformers.models.albert.modeling_albert.AlbertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [40]:
print(attack)

Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  10
    (max_iters):  5
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper__
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


In [41]:
attack_stats(attack, dataset, 25)

  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  10
    (max_iters):  5
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper__
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


[Succeeded / Failed / Total] 1 / 0 / 1:   4%|▍         | 1/25 [00:00<00:15,  1.59it/s]

[Succeeded / Failed / Total] 2 / 0 / 2:   8%|▊         | 2/25 [00:00<00:11,  2.00it/s]

[Succeeded / Failed / Total] 2 / 0 / 3:  12%|█▏        | 3/25 [00:01<00:07,  2.94it/s]

...

[Succeeded / Failed / Total] 20 / 3 / 29: : 29it [01:26,  2.97s/it]

[Succeeded / Failed / Total] 20 / 4 / 30: : 30it [01:35,  3.18s/it]

[Succeeded / Failed / Total] 21 / 4 / 31: : 31it [01:35,  3.09s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 21     |
| Number of failed attacks:     | 4      |
| Number of skipped attacks:    | 6      |
| Original accuracy:            | 80.65% |
| Accuracy under attack:        | 12.9%  |
| Attack success rate:          | 84.0%  |
| Average perturbed word %:     | 18.6%  |
| Average num. words per input: | 18.74  |
| Avg num queries:              | 134.92 |
+-------------------------------+--------+


textattack: Attack time: 95.89386868476868s


### FastText, GreedyWordSwapWIR, PartsofSpeech

In [45]:
# We're going to use our word swap class as the attack transformation.
transformation = Swapper()

constraints = [PartOfSpeech()]

search_method = GreedyWordSwapWIR()

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

textattack: Unknown if model of class <class 'transformers.models.albert.modeling_albert.AlbertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [46]:
print(attack)

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
  (is_black_box):  True
)


In [47]:
attack_stats(attack, dataset, 25)


  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
  (is_black_box):  True
)



[Succeeded / Failed / Total] 1 / 0 / 1:   4%|▍         | 1/25 [00:00<00:15,  1.52it/s]


[Succeeded / Failed / Total] 1 / 1 / 2:   8%|▊         | 2/25 [00:00<00:09,  2.46it/s]


[Succeeded / Failed / Total] 1 / 1 / 3:  12%|█▏        | 3/25 [00:00<00:06,  3.55it/s]

...


[Succeeded / Failed / Total] 17 / 6 / 29: : 29it [00:12,  2.30it/s]


[Succeeded / Failed / Total] 17 / 6 / 29: : 30it [00:13,  2.22it/s]
[Succeeded / Failed / Total] 18 / 6 / 30: : 30it [00:13,  2.22it/s]


[Succeeded / Failed / Total] 19 / 6 / 31: : 31it [00:14,  2.19it/s]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 19     |
| Number of failed attacks:     | 6      |
| Number of skipped attacks:    | 6      |
| Original accuracy:            | 80.65% |
| Accuracy under attack:        | 19.35% |
| Attack success rate:          | 76.0%  |
| Average perturbed word %:     | 18.38% |
| Average num. words per input: | 18.74  |
| Avg num queries:              | 25.24  |
+-------------------------------+--------+


textattack: Attack time: 14.13424801826477s


### Word2Vec, GreedyWordSwapWIR, PartsofSpeech

In [48]:
# We're going to use our word swap class as the attack transformation.
transformation = Swapper_()

constraints = [PartOfSpeech()]

search_method = GreedyWordSwapWIR()

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

textattack: Unknown if model of class <class 'transformers.models.albert.modeling_albert.AlbertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [49]:
print(attack)

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper_
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
  (is_black_box):  True
)


In [50]:
attack_stats(attack, dataset, 25)


  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper_
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
  (is_black_box):  True
)



[Succeeded / Failed / Total] 0 / 1 / 1:   4%|▍         | 1/25 [00:00<00:17,  1.34it/s]


[Succeeded / Failed / Total] 0 / 2 / 2:   8%|▊         | 2/25 [00:00<00:10,  2.15it/s]


[Succeeded / Failed / Total] 0 / 2 / 3:  12%|█▏        | 3/25 [00:00<00:07,  3.11it/s]

...


[Succeeded / Failed / Total] 12 / 11 / 29: : 29it [00:12,  2.32it/s]


[Succeeded / Failed / Total] 12 / 11 / 29: : 30it [00:13,  2.24it/s]
[Succeeded / Failed / Total] 12 / 12 / 30: : 30it [00:13,  2.24it/s]


[Succeeded / Failed / Total] 13 / 12 / 31: : 31it [00:13,  2.24it/s]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 13     |
| Number of failed attacks:     | 12     |
| Number of skipped attacks:    | 6      |
| Original accuracy:            | 80.65% |
| Accuracy under attack:        | 38.71% |
| Attack success rate:          | 52.0%  |
| Average perturbed word %:     | 12.39% |
| Average num. words per input: | 18.74  |
| Avg num queries:              | 24.92  |
+-------------------------------+--------+


textattack: Attack time: 13.847508668899536s


### Glove, GreedyWordSwapWIR, PartsofSpeech

In [51]:
# We're going to use our word swap class as the attack transformation.
transformation = Swapper__()

constraints = [PartOfSpeech()]

search_method = GreedyWordSwapWIR()

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

textattack: Unknown if model of class <class 'transformers.models.albert.modeling_albert.AlbertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [52]:
print(attack)

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper__
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
  (is_black_box):  True
)


In [53]:
attack_stats(attack, dataset, 25)


  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper__
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
  (is_black_box):  True
)



[Succeeded / Failed / Total] 1 / 0 / 1:   4%|▍         | 1/25 [00:00<00:10,  2.35it/s]


[Succeeded / Failed / Total] 1 / 1 / 2:   8%|▊         | 2/25 [00:00<00:07,  3.06it/s]


[Succeeded / Failed / Total] 1 / 1 / 3:  12%|█▏        | 3/25 [00:00<00:04,  4.42it/s]

...


[Succeeded / Failed / Total] 16 / 7 / 29: : 29it [00:14,  1.99it/s]


[Succeeded / Failed / Total] 16 / 7 / 29: : 30it [00:15,  1.89it/s]
[Succeeded / Failed / Total] 16 / 8 / 30: : 30it [00:15,  1.89it/s]


[Succeeded / Failed / Total] 17 / 8 / 31: : 31it [00:16,  1.90it/s]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 17     |
| Number of failed attacks:     | 8      |
| Number of skipped attacks:    | 6      |
| Original accuracy:            | 80.65% |
| Accuracy under attack:        | 25.81% |
| Attack success rate:          | 68.0%  |
| Average perturbed word %:     | 13.87% |
| Average num. words per input: | 18.74  |
| Avg num queries:              | 23.04  |
+-------------------------------+--------+


textattack: Attack time: 16.341960668563843s


### FastText, ParticleSwarmOptimization, PartsofSpeech

In [54]:
from textattack.constraints.grammaticality import PartOfSpeech
# We're going to use our word swap class as the attack transformation.
transformation = Swapper()

constraints = [PartOfSpeech()]

search_method = ParticleSwarmOptimization(pop_size=10,max_iters=5)

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

textattack: Unknown if model of class <class 'transformers.models.albert.modeling_albert.AlbertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [55]:
print(attack)

Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  10
    (max_iters):  5
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
  (is_black_box):  True
)


In [56]:
attack_stats(attack, dataset, 25)


  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  10
    (max_iters):  5
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
  (is_black_box):  True
)



[Succeeded / Failed / Total] 1 / 0 / 1:   4%|▍         | 1/25 [00:06<02:26,  6.11s/it]


[Succeeded / Failed / Total] 1 / 1 / 2:   8%|▊         | 2/25 [00:06<01:14,  3.23s/it]


[Succeeded / Failed / Total] 1 / 1 / 3:  12%|█▏        | 3/25 [00:06<00:47,  2.16s/it]

...


[Succeeded / Failed / Total] 20 / 3 / 29: : 29it [02:15,  4.67s/it]


[Succeeded / Failed / Total] 20 / 3 / 29: : 30it [02:22,  4.75s/it]
[Succeeded / Failed / Total] 21 / 3 / 30: : 30it [02:22,  4.75s/it]


[Succeeded / Failed / Total] 22 / 3 / 31: : 31it [02:29,  4.82s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 22     |
| Number of failed attacks:     | 3      |
| Number of skipped attacks:    | 6      |
| Original accuracy:            | 80.65% |
| Accuracy under attack:        | 9.68%  |
| Attack success rate:          | 88.0%  |
| Average perturbed word %:     | 19.83% |
| Average num. words per input: | 18.74  |
| Avg num queries:              | 252.0  |
+-------------------------------+--------+


textattack: Attack time: 149.52280974388123s


### Word2Vec, ParticleSwarmOptimization, PartsofSpeech

In [57]:
# We're going to use our word swap class as the attack transformation.
transformation = Swapper_()

constraints = [PartOfSpeech()]

search_method = ParticleSwarmOptimization(pop_size=10,max_iters=5)

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

textattack: Unknown if model of class <class 'transformers.models.albert.modeling_albert.AlbertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [58]:
print(attack)

Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  10
    (max_iters):  5
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper_
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
  (is_black_box):  True
)


In [59]:
attack_stats(attack, dataset, 25)


  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  10
    (max_iters):  5
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper_
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
  (is_black_box):  True
)



[Succeeded / Failed / Total] 1 / 0 / 1:   4%|▍         | 1/25 [00:04<01:50,  4.60s/it]


[Succeeded / Failed / Total] 1 / 1 / 2:   8%|▊         | 2/25 [00:04<00:55,  2.40s/it]


[Succeeded / Failed / Total] 1 / 1 / 3:  12%|█▏        | 3/25 [00:04<00:35,  1.61s/it]

...


[Succeeded / Failed / Total] 19 / 4 / 29: : 29it [01:26,  3.00s/it]


[Succeeded / Failed / Total] 19 / 4 / 29: : 30it [01:43,  3.44s/it]
[Succeeded / Failed / Total] 20 / 4 / 30: : 30it [01:43,  3.44s/it]


[Succeeded / Failed / Total] 20 / 4 / 30: : 31it [01:43,  3.34s/it]
[Succeeded / Failed / Total] 21 / 4 / 31: : 31it [01:43,  3.34s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 21     |
| Number of failed attacks:     | 4      |
| Number of skipped attacks:    | 6      |
| Original accuracy:            | 80.65% |
| Accuracy under attack:        | 12.9%  |
| Attack success rate:          | 84.0%  |
| Average perturbed word %:     | 17.34% |
| Average num. words per input: | 18.74  |
| Avg num queries:              | 193.6  |
+-------------------------------+--------+


textattack: Attack time: 103.6505913734436s


### Glove, ParticleSwarmOptimization, PartsofSpeech

In [60]:
# We're going to use our word swap class as the attack transformation.
transformation = Swapper__()

constraints = [PartOfSpeech()]

search_method = ParticleSwarmOptimization(pop_size=10,max_iters=5)

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

textattack: Unknown if model of class <class 'transformers.models.albert.modeling_albert.AlbertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [61]:
print(attack)

Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  10
    (max_iters):  5
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper__
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
  (is_black_box):  True
)


In [62]:
attack_stats(attack, dataset, 25)


  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  10
    (max_iters):  5
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper__
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
  (is_black_box):  True
)



[Succeeded / Failed / Total] 1 / 0 / 1:   4%|▍         | 1/25 [00:00<00:19,  1.25it/s]


[Succeeded / Failed / Total] 1 / 1 / 2:   8%|▊         | 2/25 [00:01<00:17,  1.28it/s]


[Succeeded / Failed / Total] 1 / 1 / 3:  12%|█▏        | 3/25 [00:01<00:11,  1.88it/s]

...


[Succeeded / Failed / Total] 18 / 5 / 28: : 29it [02:50,  5.87s/it]
[Succeeded / Failed / Total] 18 / 5 / 29: : 29it [02:50,  5.87s/it]


[Succeeded / Failed / Total] 18 / 5 / 29: : 30it [03:09,  6.31s/it]
[Succeeded / Failed / Total] 19 / 5 / 30: : 30it [03:09,  6.31s/it]


[Succeeded / Failed / Total] 19 / 5 / 30: : 31it [03:10,  6.13s/it]
[Succeeded / Failed / Total] 20 / 5 / 31: : 31it [03:10,  6.13s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 20     |
| Number of failed attacks:     | 5      |
| Number of skipped attacks:    | 6      |
| Original accuracy:            | 80.65% |
| Accuracy under attack:        | 16.13% |
| Attack success rate:          | 80.0%  |
| Average perturbed word %:     | 16.85% |
| Average num. words per input: | 18.74  |
| Avg num queries:              | 171.48 |
+-------------------------------+--------+


textattack: Attack time: 190.15324568748474s
